In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification, BertConfig
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Sentiment Analysis/emoevent_en.csv'
df = pd.read_csv(file_path, delimiter='\t')
label_encoder = LabelEncoder()
df['emotion'] = label_encoder.fit_transform(df['emotion'])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized = tokenizer(df['tweet'].tolist(), max_length=128, padding='max_length', truncation=True, return_tensors='tf')

(…)cased/resolve/main/tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

(…)bert-base-uncased/resolve/main/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)base-uncased/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)rt-base-uncased/resolve/main/config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
X = tokenized['input_ids'].numpy()
y = df['emotion'].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
config = BertConfig.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
optimizer = Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=16, class_weight=class_weights_dict, validation_split=0.1, callbacks=[early_stopping])

Epoch 1/5
329/329 [==============================] - 207s 474ms/step - loss: 1.9708 - accuracy: 0.1301 - val_loss: 1.9476 - val_accuracy: 0.0308
Epoch 2/5
329/329 [==============================] - 154s 469ms/step - loss: 1.8116 - accuracy: 0.2444 - val_loss: 1.4514 - val_accuracy: 0.4291
Epoch 3/5
329/329 [==============================] - 154s 468ms/step - loss: 1.2650 - accuracy: 0.4368 - val_loss: 1.5922 - val_accuracy: 0.4051
Epoch 4/5
329/329 [==============================] - 154s 469ms/step - loss: 0.7957 - accuracy: 0.5802 - val_loss: 1.4463 - val_accuracy: 0.4667
Epoch 5/5
329/329 [==============================] - 149s 453ms/step - loss: 0.4802 - accuracy: 0.7080 - val_loss: 1.4719 - val_accuracy: 0.4957


In [ ]:
predictions = model.predict(X_test).logits
predicted_labels = [np.argmax(p) for p in predictions]
print(classification_report(y_test, predicted_labels))

46/46 [==============================] - 17s 275ms/step
              precision    recall  f1-score   support

           0       0.18      0.42      0.25        71
           1       0.34      0.37      0.35       152
           2       0.17      0.61      0.27        28
           3       0.68      0.67      0.68       412
           4       0.68      0.37      0.48       653
           5       0.60      0.49      0.54        97
           6       0.17      0.69      0.27        48

    accuracy                           0.48      1461
   macro avg       0.40      0.52      0.41      1461
weighted avg       0.59      0.48      0.50      1461

